# Getting started


In [9]:
import sys
sys.path.insert(0, '..')
# sys.path.insert(1, '../python/')
sys.path.insert(1, 'python/')

import ROOT
import root_numpy as rnp
# import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

# ROOT.enableJSVis()
ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()

In [10]:
# %load drawingTools.py

## Histo classes: the basic functionality

All classes inheriting from `BaseHistos` are capable of retrieveing the histograms from a root file so that you can access them as members of the class



In [15]:
import python.l1THistos as histos


root_file = ROOT.TFile('/Users/cerminar/Workspace/hgcal-analysis/py3/ntuple-tools/plots1/histos_doubleele_flat1to100_PU200_eg_v150.69A.root')

# retrieve a set of plots given the name

histo_name = 'PFNFtkEmEB_all_GEN'

histo_set = histos.EGResoHistos(histo_name, root_file)

dir(histo_set)



['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'fill',
 'h_dzRes',
 'h_etaRes',
 'h_exetaRes',
 'h_exphiRes',
 'h_phiRes',
 'h_ptRes',
 'h_ptResVpt',
 'h_ptResVpt_graph',
 'h_ptResp',
 'h_ptRespVeta',
 'h_ptRespVeta_graph',
 'h_ptRespVpt',
 'h_ptRespVpt_graph',
 'name_',
 'write']

you can draw one of the histograms using the following code.

**NOTE**: in `ROOTJs` drawing only happens when you draw the canvas!

In [17]:
c = ROOT.TCanvas()
histo_set.h_etaRes.Draw()
c.Draw()


Another example: the efficiencies!
Also the class used to create the efficiency plots works along the same principle. 

It has 4 important members:
   * `h_den`: the collection of histos for the GEN level distribution of all the selected GEN particle
   * `h_num`: the collection of histos for the GEN level distribution for the selcted GEN particles actually matching the object of interest 
   * `h_eff`: the GEN matching efficiency plot obtained dividing the corresponding plots of `h_num` and `h_den` collections
   * `h_ton`: the turn-on plot for a given selection (factorizing out the GEN matching efficiency)
    

In [20]:
histo_eff = histos.HistoSetEff(histo_name, root_file)

dir(histo_eff)

dir(histo_eff.h_eff)


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'h_abseta',
 'h_energy',
 'h_eta',
 'h_fBrem',
 'h_pt',
 'h_reachedEE']

In [21]:
c = ROOT.TCanvas()
histo_eff.h_eff.h_eta.Draw()
c.Draw()


## Histo files: the basic functionality

The files are organized in folders representing the class of histograms (names as in `l1THistos.py`) and the actual histograms follow the following naming convention:

`<tp-name>_<tp-selection>_<gen-selection>_h_<histoname>`

The part up to the `_h_` is actually the name we used to read the HistoClasses in the previous examples.

The class `HistoFile` allows to browse the content of the file in a couple of lines.



In [23]:
# %load samples.py

samples = [
    HistoFile('doubleele_flat1to100_PU200_eg', 'PU200', 'v150.68A', 'eg'),
]

dir(samples[0])

print(samples[0].histo_filename)

../plots1/histos_doubleele_flat1to100_PU200_eg_v150.68A.root


For each Histo Class stored in the file you can print all the strings corresponding to the `<tp-name>`, `<tp-selection>`, `<gen-selection>`

In [4]:
# %load python/settings.py

for hf in samples:
    hf.print_primitives()



<HistoFile ../plots1/histos_doubleele_flat1to100_PU200_eg_v150.68A.root, eg>
--- HistoClass: 
['EGHistos', 'EGResoHistos', 'HistoSetEff', 'DecTkHistos', 'DecTkResoHistos']
--- TPs: 
[   'tkEleEB',
    'tkEmEE',
    'tkEmEB',
    'PFNFtkEmEE',
    'PFNFtkEleEE',
    'tkEleEE',
    'PFNFtkEmEB',
    'PFNFtkEleEB',
    'PFDecTk']
--- TP selections:
{   'PFDecTk': ['all', 'Pt10', 'Pt20'],
    'PFNFtkEleEB': [   'LooseTkIDPt10',
                       'Pt10',
                       'LooseTkID',
                       'all',
                       'Pt20',
                       'LooseTkIDPt20'],
    'PFNFtkEleEE': [   'EGq1Pt20',
                       'EGq1',
                       'EGq2Pt10',
                       'EGq2',
                       'EGq1Pt10',
                       'EGq2Pt20'],
    'PFNFtkEmEB': [   'Pt20',
                      'LooseTkIDPt20',
                      'LooseTkIDPt10',
                      'LooseTkID',
                      'all',
                      'Pt10'

## Labels for drawing

In the `ntuple-tools` all *collections*, *selections* and *samples* have a label assigned which can be used during the drawing. You can retrieve all of them via the `EventManager` which stores them in an python `dict`: the `key` is the object `name` and the `label` is in ROOT `TLatex` format. 


In [24]:
import python.collections as collections
import python.selections as selections

labels_dict = {}

evm = collections.EventManager()
labels_dict.update(evm.get_labels())
selm = selections.SelectionManager()
labels_dict.update(selm.get_labels())

print(labels_dict)





{'MC': 'MC particles', 'GEN': '', 'TC': 'Trigger Cells', 'TCTrue': 'Trigger Cells True', 'DEF2D': 'dRC2d', 'DEF2DTrue': 'dRC2d True', 'HMvDRTrue': 'HM+dR(layer) True Cl3d', 'DEF': 'dRC3d', 'DEFNC': 'dRC3d NewTh', 'HMvDR': 'HM+dR(layer) Cl3d', 'HMvDREmInt': 'HM+dR(layer) Cl3d EM Int', 'HMvDREmIntMerged': 'HM+dR(layer) Cl3d EM Int Merged', 'HMvDRRebin': 'HM+dR(layer) rebin Cl3d ', 'HMvDRsTC': 'HM+dR(layer) SuperTC Cl3d ', 'HMvDRNC0': 'HM+dR(layer) Cl3d + NewTh0', 'HMvDRNC1': 'HM+dR(layer) Cl3d + NewTh1', 'DEFMerged': 'dRC3d merged', 'DEFCalib': 'dRC3d calib.', 'HMvDRMerged': 'HM+dR(layer) merged', 'HMvDRfixed': 'HM fixed', 'HMvDRcylind10': 'HM Cylinder 10cm', 'HMvDRcylind5': 'HM Cylinder 5cm', 'HMvDRcylind2p5': 'HM Cylinder 2.5cm', 'HMvDRshape': 'HM shape', 'HMvDRshapeDr': 'HM #Delta#rho < 0.015', 'HMvDRshapeDtDu': 'HM #Deltau #Deltat', 'HMvDRCalib': 'HM calib.', 'HMvDRcylind10Calib': 'HM Cylinder 10cm calib.', 'HMvDRcylind5Calib': 'HM Cylinder 5cm calib.', 'HMvDRcylind2p5Calib': 'HM Cyl

## Automatic reading of the histograms

The `HPlot` class allows to read all the histos belonging to a certain HistoClass in a `list` of samples.

In [25]:
%%time

import python.l1THistos as histos

hplot = HPlot(samples, labels_dict)

hplot.create_histo_proxies(histos.EGResoHistos)
hplot.create_histo_proxies(histos.DecTkResoHistos)


--- EGHistos
# of plots: 1664
# of primitives: 208
--- EGResoHistos
# of plots: 2080
# of primitives: 208
--- GenParticleHistos
# of plots: 2640
# of primitives: 220
--- DecTkHistos
# of plots: 192
# of primitives: 12
--- DecTkResoHistos
# of plots: 180
# of primitives: 12
--- EGHistos
# of plots: 1664
# of primitives: 208
--- EGResoHistos
# of plots: 2080
# of primitives: 208
--- GenParticleHistos
# of plots: 2640
# of primitives: 220
--- DecTkHistos
# of plots: 192
# of primitives: 12
--- DecTkResoHistos
# of plots: 180
# of primitives: 12
CPU times: user 642 ms, sys: 52.5 ms, total: 694 ms
Wall time: 714 ms


The histogram classes are stored in a pandas `DataFrame`

In [26]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,eg,PU200,PFNFtkEleEE,EGq1Pt10,GENEeEtaBC,<class 'python.l1THistos.EGResoHistos'>,<drawingTools.HProxy object at 0x13b6eacd0>
1,eg,PU200,tkEleEB,LooseTkID,GENPt15,<class 'python.l1THistos.EGResoHistos'>,<drawingTools.HProxy object at 0x127c6c5e0>
2,eg,PU200,tkEmEE,EGq4,GENEeEtaBC,<class 'python.l1THistos.EGResoHistos'>,<drawingTools.HProxy object at 0x127bec370>
3,eg,PU200,PFNFtkEleEE,EGq1Pt20,GENEePt30,<class 'python.l1THistos.EGResoHistos'>,<drawingTools.HProxy object at 0x127c6c580>
4,eg,PU200,tkEmEE,EGq4Pt20,GENEe,<class 'python.l1THistos.EGResoHistos'>,<drawingTools.HProxy object at 0x127bec430>
5,eg,PU200,tkEmEB,LooseTkID,GENPt15,<class 'python.l1THistos.EGResoHistos'>,<drawingTools.HProxy object at 0x127bca460>
6,eg,PU200,tkEleEE,EGq5Pt30,GENEe,<class 'python.l1THistos.EGResoHistos'>,<drawingTools.HProxy object at 0x13b6eabe0>
7,eg,PU200,PFNFtkEleEB,LooseTkIDPt20,GENEtaF,<class 'python.l1THistos.EGResoHistos'>,<drawingTools.HProxy object at 0x127c52fd0>
8,eg,PU200,PFNFtkEleEB,all,GENEtaF,<class 'python.l1THistos.EGResoHistos'>,<drawingTools.HProxy object at 0x127c65790>
9,eg,PU200,tkEmEB,LooseTkIDPt10,GENEtaF,<class 'python.l1THistos.EGResoHistos'>,<drawingTools.HProxy object at 0x13b6eaa90>


The `HPlot` class allows to retrieve a set histograms belonging to a certain HistoClass on the basis of the `<tp-name>`, `<tp-selection>`, `<gen-selection>`

In [32]:
hsets, labels, text = hplot.get_histo(histos.EGResoHistos, 'eg', ['PU200'], 'PFNFtkEmEE', 'EGq1Pt10', 'GENEeEtaBC', False)            
print(hsets)
print(labels)
print(text)

dir(hsets[0])


[<EGResoHistos PFNFtkEmEE_EGq1Pt10_GENEeEtaBC>]
['']
PU200, TkEm EE Corr. New, hwQual=1, p_{T}^{L1}>=10GeV, 1.52 < |#eta^{GEN}| <= 2.4


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'fill',
 'h_etaRes',
 'h_exetaRes',
 'h_exphiRes',
 'h_hdzRes',
 'h_phiRes',
 'h_ptRes',
 'h_ptResVpt',
 'h_ptResVpt_graph',
 'h_ptResp',
 'h_ptRespVeta',
 'h_ptRespVeta_graph',
 'h_ptRespVpt',
 'h_ptRespVpt_graph',
 'name_',
 'write']

In [33]:
c1 = ROOT.TCanvas()
hsets[0].h_ptRes.Draw('hist')
c1.Draw()

## DrawingTools

Once you retrieved the set of histograms, labels and text from the file you can produce the plot using the drawing tools.


In [34]:
dm = DrawMachine(rleg_config)

dm.addHistos([his.h_exphiRes for his in hsets], labels)

dm.draw(text=text, options='hist', norm=True)


In [36]:
help(dm.draw)

Help on method draw in module drawingTools:

draw(text, options='', norm=False, y_log=False, x_log=False, y_min=None, y_max=None, x_min=None, x_max=None, y_axis_label=None, x_axis_label=None, v_lines=None, h_lines=None, do_profile=False, do_ratio=False, y_min_ratio=0.7, y_max_ratio=1.3) method of drawingTools.DrawMachine instance

